In [1]:
using Random
using Plots

In [ ]:
mutable struct Particle
    x::Int64
    y::Int64
    
end;

In [82]:
# Function to initialize the particles on the lattice
function initialize_particles!(lattice, N)
    Random.seed!(1234)
    Lx,Ly = size(lattice)
    idx = shuffle(collect(1:Lx*Ly))[1:N]
    lattice[idx] .= 1
end;

In [58]:
# Let's say you have a 2D array
A = [1 4; 2 5; 3 6]  # 3x2 array

# And you have a single index
i = 4

# You can access the element at that index with:
element = A[i]

println("The element at index $i is $element.")

The element at index 4 is 4.


In [29]:
function calculate_active_particles(lattice)
    active_particles = 0
    L = size(lattice, 1) # Assuming lattice is a square
    for i in 1:L
        for j in 1:L
            if lattice[i, j] == 1
                # Check the four neighboring sites using periodic boundary conditions
                neighbors = [(i % L) + 1, (i-2 % L) + 1, j, (j % L) + 1, (j-2 % L) + 1]
                if any(lattice[neighbors[k]] == 1 for k in 1:4)
                    active_particles += 1
                end
            end
        end
    end
    return active_particles
end;

In [523]:
function NearestNeighbors(lat, x,y)
    Lx, Ly = size(lat)
    nn = zeros(Int64, 4)
    
    xp = mod(x, Lx) + 1
    yp = mod(y, Ly) + 1
    xm = mod(x - 2, Lx) + 1
    ym = mod(y - 2, Ly) + 1
    
    nn[1] = PointToIndex(lat, Point2D(xm, y)) # Left neighbor
    nn[2] = PointToIndex(lat, Point2D(xp, y)) # Right neighbor
    nn[3] = PointToIndex(lat, Point2D(x, ym)) # Upper neighbor
    nn[4] = PointToIndex(lat, Point2D(x, yp)) # Lower neighbor
    
    return nn
end;

In [30]:
function update_lattice!(lattice)
    L = size(lattice, 1) # Assuming lattice is a square
    for i in 1:L
        for j in 1:L
            if lattice[i, j] == 1
                # Check the four neighboring sites using periodic boundary conditions
                neighbors = [(i % L) + 1, (i-2 % L) + 1, j, (j % L) + 1, (j-2 % L) + 1]
                neighbors = NearestNeighbors
                if any(lattice[neighbors[k]] == 1 for k in 1:4)
                    # Find empty neighbors
                    empty_neighbors = [k for k in neighbors if lattice[k] == 0]
                    if !isempty(empty_neighbors)
                        # Choose a random empty neighbor
                        new_site = rand(empty_neighbors)
                        # Move the particle to the new site
                        lattice[i, j] = 0
                        lattice[new_site] = 1
                    end
                end
            end
        end
    end
end;

In [31]:
# Function to run the simulation
function run_simulation(lattice, N, steps)
    initialize_particles!(lattice, N)
    for t in 1:steps
        update_lattice!(lattice)
    end
    return calculate_active_particles(lattice)
end;

In [520]:
# Define the lattice size and dimension
L = 64
d = 2

# Initialize the lattice
lattice = zeros(Int, L, L);

In [521]:
# Run the simulation for different densities
densities = range(0, stop=1, length=100)
active_fractions = [run_simulation(lattice, round(Int, ϕ * L^d), 1000) / (ϕ * L^d) for ϕ in densities];

LoadError: BoundsError: attempt to access 64×64 Matrix{Int64} at index [0]

In [ ]:
# Plot the active fraction as a function of density
plot(densities, active_fractions)